In [15]:
import os, sys
from PIL import Image
from PIL.ExifTags import TAGS




            
# Fonction pour récupérer les métadonnées d'une image
def obtenir_métadonnées_image(chemin_image):
    try:
        with open(chemin_image, 'rb') as f:
            exif_data = chemin_image._getexif(f)
            for TAGS in exif_data.items():
                if tag in TAGS:
            
                    métadonnées = {
                    'chemin_image': chemin_image,
                    'taille': os.path.getsize(chemin_image),
                    'format': os.path.splitext(chemin_image)[1],
                    'orientation': tag.get('Image Orientation', 'Non spécifié'),
                    'date_creation': tag.get('DateTimeOriginal', 'Non spécifiée'),
                    'modèle_appareil': tag.get('Model', 'Non spécifié')
                    # Ajoutez d'autres métadonnées Exif pertinentes ici
                  }
            return métadonnées
    except Exception as e:
        print(f"Erreur lors de la récupération des métadonnées de l'image {chemin_image}: {e}")
        return None

# Dossier contenant les images
dossier_images = 'images'

# Parcourir le dossier contenant les images et récupérer les métadonnées de chaque image
métadonnées_images = []
for nom_fichier in os.listdir(dossier_images):
    chemin_image = os.path.join(dossier_images, nom_fichier)
    if os.path.isfile(chemin_image):
        métadonnées = obtenir_métadonnées_image(chemin_image)
        if métadonnées:
            métadonnées_images.append(métadonnées)

# Enregistrer les métadonnées dans un fichier JSON
with open('metadonnees_images.json', 'w') as f:
    json.dump(métadonnées_images, f, indent=4)

print("Métadonnées des images enregistrées avec succès dans 'metadonnees_images.json'.")


Erreur lors de la récupération des métadonnées de l'image images/tuqa-nabi-uhnZZUaTIOs-unsplash.jpg: 'str' object has no attribute '_getexif'
Erreur lors de la récupération des métadonnées de l'image images/linh-nguyen-7mRX9WRDAHQ-unsplash.jpg: 'str' object has no attribute '_getexif'
Erreur lors de la récupération des métadonnées de l'image images/danilo-batista-eSceitc-s30-unsplash.jpg: 'str' object has no attribute '_getexif'
Erreur lors de la récupération des métadonnées de l'image images/sindy-strife-qHrp2ZskqYU-unsplash.jpg: 'str' object has no attribute '_getexif'
Erreur lors de la récupération des métadonnées de l'image images/slavy-darozhkin-O_UbPKaz6no-unsplash.jpg: 'str' object has no attribute '_getexif'
Erreur lors de la récupération des métadonnées de l'image images/anton-kraev-TuU5tODcrzU-unsplash.jpg: 'str' object has no attribute '_getexif'
Erreur lors de la récupération des métadonnées de l'image images/lisheng-chang-xNSy-b_YfBY-unsplash.jpg: 'str' object has no attri

Visualisation des données 

In [ ]:
from pandas import json_normalize
import pandas as pd
import json
import matplotlib.pyplot as plot

#data = json.load(open("../../data/pl.json"))
dataframe = json_normalize(data)
grouped = dataframe.groupby("year").count()
plot.plot(grouped)
plot.show()

